In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/adult-census-income/adult.csv")


df.head()

# Cleaning the dataset

In [ ]:
df["income"] = df["income"].str.strip()
df["income"] = df["income"].map({"<=50K": 0, ">50K": 1})
print("Converted the target from string to a binary")

In [ ]:

df = df.replace("?", np.nan)
df = df.dropna()

print("Removing the nan rows")

# Preparing the model

In [ ]:
X = df.drop(columns=["income", "fnlwgt"])
y = df["income"]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [ ]:
categorical_features = X_train.select_dtypes(include="object").columns
numerical_features = X_train.select_dtypes(exclude="object").columns


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


# Bais detection

In [ ]:
sensitive_feature = X_test["sex"]
sensitive_feature.value_counts()


In [ ]:
!pip install fairlearn

from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    true_positive_rate,
    false_positive_rate
)


In [ ]:

metric_frame = MetricFrame(
    metrics={
        "selection_rate": selection_rate,
        "true_positive_rate": true_positive_rate,
        "false_positive_rate": false_positive_rate,
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)


In [ ]:
metric_frame.by_group


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

metric_frame_extended = MetricFrame(
    metrics={
        "precision": precision_score,
        "recall": recall_score,
        "f1_score": f1_score,
        "selection_rate": selection_rate,
        "true_positive_rate": true_positive_rate,
        "false_positive_rate": false_positive_rate,
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

# Overall metrics
print("Overall Metrics:")
print(metric_frame_extended.overall)

# Metrics by group
print("\nMetrics by Group:")
print(metric_frame_extended.by_group)


In [ ]:
print("\nMetric Differences Between Groups:")
print(metric_frame_extended.difference())


In [ ]:
import matplotlib.pyplot as plt

# Use the metrics by group
metrics_by_group = metric_frame_extended.by_group

# Transpose so metrics are columns
metrics_by_group = metrics_by_group.T


metrics = metrics_by_group.index
groups = metrics_by_group.columns

fig, axes = plt.subplots(nrows=len(metrics), ncols=1, figsize=(6, 4*len(metrics)))

for i, metric in enumerate(metrics):
    axes[i].bar(groups, metrics_by_group.loc[metric])
    axes[i].set_title(metric)
    axes[i].set_ylim(0, 1)
    axes[i].set_ylabel("Value")
    
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score

# Create a MetricFrame for accuracy
accuracy_frame = MetricFrame(
    metrics=accuracy_score,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_feature
)

print("Overall Accuracy:", accuracy_frame.overall)
print("\nAccuracy by Sex:")
print(accuracy_frame.by_group)
